In [1]:
import json
import glob
import pandas as pd
import numpy as np
import sklearn
from numpy import asarray
from PIL import Image
import torch
from torchvision.transforms import ToTensor
from torch import flatten
import os
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_curve

data = '/Users/paddy/Desktop/AiCore/facebook_ml/final_dataset/image_data.csv'
images = '/Users/paddy/Desktop/AiCore/facebook_ml/images_raw'
images_corrected = '/Users/paddy/Desktop/AiCore/facebook_ml/Images'

In [2]:
image_list = glob.glob(f'{images}/*.jpg')
df = pd.read_csv(data)
df = df.drop(['Unnamed: 0', 'bucket_link', 'image_ref', 'create_time'], axis=1)

df

,uuid,id,price,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles,id.1
0,912bb259-3ad9-457b-9db1-ce1da9016057,5f5f57d7-778f-4336-bb10-b43863418c8c,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,243809c0-9cfc-4486-ad12-3b7a16605ba9
1,b166d305-b852-4bdd-83f4-465b20da94fa,5f5f57d7-778f-4336-bb10-b43863418c8c,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,dcbe214e-cb4b-4f18-ab16-218ec12ab68b
2,68f5a29d-0075-4d60-81c1-ab684a82e50c,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,6f228cd8-a2d6-4f23-858b-4a77a2f42d7c
3,f6a309d7-d247-446a-9b5e-aceefdd4334d,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2275f5f2-ee51-410c-8d85-e5923285a864
4,2c2b3a6f-15b3-4289-937a-15482d9f5781,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,5.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,c0e5ef79-09c0-4cb7-9b7c-edd3d53aa530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,e2d7e9e4-8a94-4293-a7a4-e4209e3c9243,e95a25e7-f2be-4ca0-aabb-a3a3642d2cc6,55.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,a9e31ee1-c706-42a9-97d9-4ca257e4c6f3
6307,4efda54a-d4e6-4d4f-8d59-dc1d9a3bbd58,0276f1d7-570a-4420-832a-d1a13c487c0a,88.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,f3bcf410-3972-4021-bc0f-dae1892f8793
6308,4be61860-85ad-4217-aacb-a3729176cad0,0276f1d7-570a-4420-832a-d1a13c487c0a,63.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,dd1a90ff-4a1f-4ff8-8183-381d79447bf2
6309,53e142a5-b293-47a0-aff8-cd0db1cd291f,5083e833-e273-4ca9-b68f-a3cbb1bfeaf2,179.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,ebe8d613-4aa0-477c-8f1a-256ff8d4f248


In [3]:
df_for_model = df.drop(['uuid', 'price', 'id.1'], axis=1)
df_for_model

,id,category_Baby & Kids Stuff,"category_Clothes, Footwear & Accessories",category_Computers & Software,category_DIY Tools & Materials,category_Health & Beauty,category_Home & Garden,"category_Music, Films, Books & Games",category_Office Furniture & Equipment,category_Other Goods,"category_Phones, Mobile Phones & Telecoms","category_Sports, Leisure & Travel",category_Video Games & Consoles
0,5f5f57d7-778f-4336-bb10-b43863418c8c,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5f5f57d7-778f-4336-bb10-b43863418c8c,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,c2c8949f-3cde-4651-a234-4a4a1b2a9ad4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8292aa4e-7f1b-4655-bf0e-f1f2c9e3ffaf,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6306,e95a25e7-f2be-4ca0-aabb-a3a3642d2cc6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6307,0276f1d7-570a-4420-832a-d1a13c487c0a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6308,0276f1d7-570a-4420-832a-d1a13c487c0a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6309,5083e833-e273-4ca9-b68f-a3cbb1bfeaf2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
# df.columns
column_indicies = [1,2,3,4,5,6,7,8,9,10,11,12]
new_column = ['0','1','2','3','4','5','6','7','8','9','10','11']
old_names = df_for_model.columns[column_indicies]
df_for_model.rename(columns=dict(zip(old_names, new_column)),inplace=True)
df_for_model = df_for_model.drop(['id'], axis=1)

df_for_model

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6306,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [43]:
# df_pls_work = df_for_model[df_for_model != 0].stack().reset_index(level=0, drop=True)
df_dict_maybe = df_for_model.to_list()
df_dict_maybe

AttributeError: 'DataFrame' object has no attribute 'to_list'

In [39]:

df_pls_work = df_for_model[df_for_model != 0].stack().reset_index(level=0, drop=True)
# print(type(df_pls_work))
dict_final = df_pls_work.to_dict()
dict_final
# type(dict_pls)

{'5': 1.0,
 '0': 1.0,
 '1': 1.0,
 '8': 1.0,
 '10': 1.0,
 '3': 1.0,
 '6': 1.0,
 '2': 1.0,
 '9': 1.0,
 '4': 1.0,
 '7': 1.0,
 '11': 1.0}

{0: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 1: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 2: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 3: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 4: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 5: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 6: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 7: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 8: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 9: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 10: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11'], 11: ['5', '0', '1', '8', '10', '3', '6', '2', '9', '4', '7', '11']}


In [7]:
class image_setup:
    def __init__(self):
        self.df = df
        self.image_list_fixed = glob.glob(f'{images_corrected}/*.jpg')
        self.image_id_list = []
        self.class_list = []
        self.flattened = []
        self.image_url_to_df()
        self.class_number()
        self.image_correct_prop()
        self.image_to_tensor()
        self.training_tuples()

    def image_url_to_df(self):
        for img in image_list[:50]:
            # print(os.path.basename(img[:-4]))
            image_id = pd.DataFrame({ 'image' : [os.path.basename(img[:-4])]})
            self.image_id_list.append(image_id)
            # print(self.image_id_list)
        return self.image_id_list

    def class_number(self):
        class_number = self.df.apply(lambda row: row[row == 1.0].index, axis=1)
        self.class_list.append(class_number)
        print(self.class_list)
        
        return self.class_list


    def image_correct_prop(self):
        resized_img = 128
        for n, img in enumerate(image_list[:50], 1):
            image_ite = Image.open(img)
            background = Image.new(mode='RGB', size=(resized_img, resized_img)) 
            original_img = image_ite.size
            max_dim = max(image_ite.size)
            ratio = resized_img / max_dim
            img_corr_ratio =(int(original_img[0] * ratio), int(original_img[1] * ratio))
            img_corr = image_ite.resize(img_corr_ratio)
            # print(img_corr)
            background.paste(img_corr, (((resized_img - img_corr_ratio[0]) // 2), ((resized_img - img_corr_ratio[1]) // 2)))
            # print(background)
            background.save(f'{images_corrected}/{img[50:]}')

    
    def image_to_tensor(self):
        for i, img in enumerate(image_list_f[:50], 1):
            image = Image.open(img)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            # print(self.flattened)
            self.flattened.append(flattened)

        return self.flattened

    def training_tuples(self):
        self.ready_tuple = zip(self.flattened, self.class_list)
        print(self.ready_tuple)


            
    
    pass
image_setup()

KeyboardInterrupt: 

In [21]:
image_array = []
image_list_f = glob.glob(f'{images_corrected}/*.jpg')
def image_to_tensor():
        for img in image_list_f[:50]:
            image = Image.open(img)
            # print(image)
            t = ToTensor()
            tensor = t(image)
            flattened = torch.flatten(tensor)
            flatten_numpy = flattened.numpy()
            # print(flattened.dtype)
            image_array.append(flatten_numpy)

            # print(empty)
        return image_array
image_to_tensor()
print(len(image_array))


50


In [27]:
list_of_tuples = []
list_of_classes = []
for key, value in dict_final.items():
        list_of_classes.extend(key)
# print(list_of_classes)
# print(image_array[0].shape)
# print(len(fixed.values))
print(len(list_of_classes))
def training_tuples():
        for i in range(len(image_array)):
                # print(image_array[i])
                # print(image_array[i])
                ready_tuple = (image_array[i], list_of_classes[i])
                list_of_tuples.append(ready_tuple)
        return list_of_tuples
training_tuples()
# list_of_tuples[:5].shape

14


IndexError: list index out of range

In [16]:
print(list_of_tuples[1])

IndexError: list index out of range

In [ ]:
print(image_array[1].shape)
size = 128*128*3
n = len(image_array)
print(len(image_array))
print(image_array[1])


torch.Size([49152])
50
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [ ]:
# m = no of labels
#  features = 49152 = items in list of flattened image :)
m = 12 
sh = image_array[0].shape


X = np.zeros((m, 49152))
y = np.zeros(m)
print(X.shape, y.shape)
for item in range(len(image_array)):
    features, labels = image_array[item]
    print(features.shape)
    X[item :] = features
    y[item] = labels
    

(12, 49152) (12,)


TypeError: list indices must be integers or slices, not tuple

In [ ]:
model = LogisticRegression()
model.fit(X, y)
pred = model.predict(X)
print ('acc:', accuracy_score(y, pred))
print('prec:', precision_recall_curve(y, pred))

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 0.0